In [2]:
#test 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").getOrCreate()

df = spark.createDataFrame([
    (1, "Sara"), (2, "Souhail")
], ["id", "name"])

df.show()

spark.stop()


25/07/08 01:30:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

+---+-------+
| id|   name|
+---+-------+
|  1|   Sara|
|  2|Souhail|
+---+-------+



In [2]:
from kafka import KafkaProducer
import json
from pyspark.sql import SparkSession
from pyspark.ml.linalg import DenseVector

# Spark Session
spark = SparkSession.builder \
    .appName("KafkaProducerOnlyFeatures") \
    .getOrCreate()

df_final = spark.read.parquet("data/df_final.parquet").dropna()

# Initialiser Kafka Producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

# Envoyer uniquement les valeurs du vecteur features
for row in df_final.select("features").toLocalIterator():
    feature_vector = row["features"]
    
    # Convertir le vecteur en une liste
    if isinstance(feature_vector, DenseVector):
        features_list = feature_vector.toArray().tolist()
    else:
        features_list = list(feature_vector)

    message = {"features": {"type": 1, "values": features_list}}
    producer.send("accidents-stream", value=message)

producer.flush()
producer.close()
print("✅ Données (features uniquement) envoyées avec succès !")


25/07/08 02:22:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/07/08 02:22:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 27:==========================================================(1 + 0) / 1]

✅ Données (features uniquement) envoyées avec succès !
